In [1]:
import pandas as pd
import requests as rq
import json
import league_data_fetcher

In [ ]:
response_raw = rq.get('https://fantasy.premierleague.com/api/element-summary/405/')
response: json = response_raw.json()

In [ ]:
url_bootstrap = 'https://fantasy.premierleague.com/api/bootstrap-static/'
bootstrap_raw = rq.get(url_bootstrap)

In [8]:
league_response_raw = rq.get('https://fantasy.premierleague.com/api/entry/21482/event/1/picks/')
league_response: json = league_response_raw.json()

In [ ]:
fpl = league_data_fetcher.Fetcher(1.1, 27964, 1)

Fetching league entries
Entry #1 saved, sleeping for 1.1 s
Entry #2 saved, sleeping for 1.1 s
Entry #3 saved, sleeping for 1.1 s
Entry #4 saved, sleeping for 1.1 s
Entry #5 saved, sleeping for 1.1 s
Entry #6 saved, sleeping for 1.1 s
Entry #7 saved, sleeping for 1.1 s
Entry #8 saved, sleeping for 1.1 s
Entry #9 saved, sleeping for 1.1 s
Entry #10 saved, sleeping for 1.1 s
Entry #11 saved, sleeping for 1.1 s
Entry #12 saved, sleeping for 1.1 s
Entry #13 saved, sleeping for 1.1 s
Entry #14 saved, sleeping for 1.1 s
Entry #15 saved, sleeping for 1.1 s
Entry #16 saved, sleeping for 1.1 s
Entry #17 saved, sleeping for 1.1 s
Entry #18 saved, sleeping for 1.1 s
Entry #19 saved, sleeping for 1.1 s
Entry #20 saved, sleeping for 1.1 s
Entry #21 saved, sleeping for 1.1 s
Entry #22 saved, sleeping for 1.1 s
Entry #23 saved, sleeping for 1.1 s
Entry #24 saved, sleeping for 1.1 s
Entry #25 saved, sleeping for 1.1 s
Entry #26 saved, sleeping for 1.1 s
Entry #27 saved, sleeping for 1.1 s
Entry #28 sav

In [3]:
fpl.team_dict[21482].aggregate('bps', 'median')

16

In [5]:
fpl.team_dict[82846].uniqueness

0.65